In [1]:
import pandas as pd
import requests
import json
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
data = requests.get('https://e47c-115-245-99-238.ngrok-free.app/api/mindmate/get-all')
data = json.loads(data.text)
df = pd.DataFrame(data)
df.head()

,_id,name,email,profile,password,expertise,mates,__v,anonymous,bio
0,666c117cbbc78a6be01797b4,Anekant Jain,temp1@mindbridge.com,https://cdn.pixabay.com/photo/2015/10/05/22/37...,mindbridge1,"[Psychologist, Depression, Anxiety, Relationsh...",[],0,Candy Crush,Dedicated psychologist with a passion for empo...
1,666c12bebbc78a6be01797b8,Ashish Prajapati,temp2@mindbridge.com,https://cdn.pixabay.com/photo/2015/10/05/22/37...,mindbridge2,"[Psychology, Anxiety Issues, Relationship Issu...",[],0,Kakashi Hatake,Dedicated psychologist with a passion for empo...
2,666c12f0bbc78a6be01797ba,Arshpreet Singh,temp3@mindbridge.com,https://cdn.pixabay.com/photo/2015/10/05/22/37...,mindbridge3,"[Psychology, Anxiety Issues, Personality Devel...",[],0,Gullu,I contribute to fostering a compassionate comm...
3,666c1307bbc78a6be01797bc,Aanya Joshi,temp4@mindbridge.com,https://cdn.pixabay.com/photo/2015/10/05/22/37...,mindbridge4,"[Anxiety, Behaviour Modifications, Emotional D...",[],0,Inner light,Dedicated psychologist with a passion for empo...
4,666c131ebbc78a6be01797be,Priya Jain,temp5@mindbridge.com,https://cdn.pixabay.com/photo/2015/10/05/22/37...,mindbridge5,"[Child and Adolescent Counseling, Grief and Tr...",[],0,Sakura,A Counseling Psychologist with experience in f...


In [3]:
# df.drop(df.columns.difference(['id','bio','expertise']),axis=1, inplace=True)

df = df[['_id','bio','expertise']]
df.head()

,_id,bio,expertise
0,666c117cbbc78a6be01797b4,Dedicated psychologist with a passion for empo...,"[Psychologist, Depression, Anxiety, Relationsh..."
1,666c12bebbc78a6be01797b8,Dedicated psychologist with a passion for empo...,"[Psychology, Anxiety Issues, Relationship Issu..."
2,666c12f0bbc78a6be01797ba,I contribute to fostering a compassionate comm...,"[Psychology, Anxiety Issues, Personality Devel..."
3,666c1307bbc78a6be01797bc,Dedicated psychologist with a passion for empo...,"[Anxiety, Behaviour Modifications, Emotional D..."
4,666c131ebbc78a6be01797be,A Counseling Psychologist with experience in f...,"[Child and Adolescent Counseling, Grief and Tr..."


In [4]:
def temp(arr):
  str = ''
  for i in arr:
    str += i + ' '
  return str

def method(str, str1):
  return str + str1

def tokenizer(str):
  tokens = word_tokenize(str)
  tokens = [word.lower() for word in tokens if word.isalpha() and word.lower() not in stopwords.words('english')]
  preprocessed_text = ' '.join(tokens)
  return preprocessed_text

In [5]:
df['expertise'] = df['expertise'].apply(temp)
df['expertise'][:2]

0    Psychologist Depression Anxiety Relationship P...
1    Psychology Anxiety Issues Relationship Issues ...
Name: expertise, dtype: object

In [6]:
tmp = pd.DataFrame(df['_id'])
tmp['desc'] = df[['bio','expertise']].apply(lambda df : method(df['bio'], df['expertise']), 1)
tmp['tokens'] = tmp['desc'].apply(tokenizer)
# tmp['tokens'][0]
tmp.head()

,_id,desc,tokens
0,666c117cbbc78a6be01797b4,Dedicated psychologist with a passion for empo...,dedicated psychologist passion empowering indi...
1,666c12bebbc78a6be01797b8,Dedicated psychologist with a passion for empo...,dedicated psychologist passion empowering indi...
2,666c12f0bbc78a6be01797ba,I contribute to fostering a compassionate comm...,contribute fostering compassionate community s...
3,666c1307bbc78a6be01797bc,Dedicated psychologist with a passion for empo...,dedicated psychologist passion empowering indi...
4,666c131ebbc78a6be01797be,A Counseling Psychologist with experience in f...,counseling psychologist experience facilitatin...


In [7]:
df_comb = pd.DataFrame(df['_id'])

df_comb['comb'] = df[['bio','expertise']].apply(lambda df : method(df['bio'], df['expertise']), 1)
# df_c.drop(['bio','expertise'], axis = 1, inplace = True)

df_comb['tokens'] = df_comb['comb'].apply(tokenizer)
df_comb.drop(['comb'],  axis = 1, inplace = True)

df_comb.head()

,_id,tokens
0,666c117cbbc78a6be01797b4,dedicated psychologist passion empowering indi...
1,666c12bebbc78a6be01797b8,dedicated psychologist passion empowering indi...
2,666c12f0bbc78a6be01797ba,contribute fostering compassionate community s...
3,666c1307bbc78a6be01797bc,dedicated psychologist passion empowering indi...
4,666c131ebbc78a6be01797be,counseling psychologist experience facilitatin...


In [8]:
df_exp = pd.DataFrame(df['_id'])
df_exp['exp'] = df['expertise']
df_exp['tokens'] = df_exp['exp'].apply(tokenizer)
df_exp.drop(['exp'],  axis = 1, inplace = True)
df_exp.head()


,_id,tokens
0,666c117cbbc78a6be01797b4,psychologist depression anxiety relationship p...
1,666c12bebbc78a6be01797b8,psychology anxiety issues relationship issues ...
2,666c12f0bbc78a6be01797ba,psychology anxiety issues personality developm...
3,666c1307bbc78a6be01797bc,anxiety behaviour modifications emotional dist...
4,666c131ebbc78a6be01797be,child adolescent counseling grief trauma mood ...


In [9]:
import pickle

In [10]:
pickle.dump(df_comb, open('df_combined.pkl', 'wb'))
pickle.dump(df_exp, open('df_expertise.pkl', 'wb'))